In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math as math
import scipy
from scipy.stats import norm

In [2]:
def histogramCalc(data, small_length, length): 
    data.columns
    a = np.array(data['X at Max Y(1)'])
    b = np.array(data['X at Max Y(4)'])

    argument = np.where(a == max(a))
    c = np.delete(a, argument)
    argument = np.where(b == max(b))
    d = np.delete(b, argument)

    e = []
    for i in range(len(c)):
        e.append(c[i] - d[i])

    e = np.array(e)

    plt.xlabel("DELAY. SGB-BCF92. " + str(small_length) + "mm")
    plt.ylabel("Frequency")
    x = np.random.gamma(4, 0.5, 1000)
    plt.axvline(e.mean(),color='k', linestyle='dashed', linewidth=1)
    mu, std = norm.fit(e)
    plt.hist(e, bins = 'auto', density=True, alpha=0.6, color='b')

    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
  
    plt.plot(x, p, 'k', linewidth=2)
    title = "Fit Values: {:.2f} and {:.2f}".format(mu, std)

    plt.show()

    mean_rounded = "{:.3f}".format((np.mean(e)) * 10 ** 9)
    sr_rounded = "{:.3f}".format((np.std(e)) * 10 ** 9)
    median_rounded = "{:.3f}".format((np.median(e)) * 10 ** 9)
    
    mean = "Mean: " + str(mean_rounded) + " ns"
    SD = "Standard Deviation: " + str(sr_rounded) + " ns"
    median = "Median: " + str(median_rounded) + " ns"
    
    print(mean)
    print(SD)
    print(median)

In [3]:
def get_all_values(data):
    delay = np.array(data['X at Max Y(1)'])
    L = []
    for val in delay:
        if val not in L:
            L.append(val)
    
    L.sort()

    return L

In [4]:
def histogram_justCalc(str1, str2, small_lengths, length):
    dic = {}
    i = 2
    for small_length in small_lengths:
        data_name = str1 + str(i) + str2
        # print(data_name)
        data = pd.read_csv(data_name, low_memory=False)
        data.columns
        a = np.array(data['X at Max Y(1)'])
        b = np.array(data['X at Max Y(4)'])

        argument = np.where(a == max(a))
        c = np.delete(a, argument)
        argument = np.where(b == max(b))
        d = np.delete(b, argument)

        e = []
        j = 0
        for j in range(len(c) - 1):
            e.append(c[j] - d[j])

        e = np.array(e)


        mean = "{:.3f}".format((np.mean(e)) * (10**9))
        SD = "{:.3f}".format((np.std(e)) * (10**9))
        median = "{:.3f}".format((np.median(e)) * (10**9))
        
        
        
        distance = length - 2*(small_length)
        
        spofl = 299792458
        # velocity = ((distance/1000) / ((np.mean(a)) - offset))
        velocity = ((distance/1000) / ((np.mean(e))))
        percentage = abs((velocity / spofl) * 100)

        dic[small_length] = [mean, SD, median, percentage]
        i += 1
    return dic

In [5]:
def offset(d, midvalue, distance):
    for k in d.keys():
        # print(d[k])
        d[k][0] = float(d[k][0]) - d[midvalue][0]
        if (d[k][0] != 0):
            d[k][3] = abs((((distance - 2*k) / d[k][0]) / 299792458)*10000000000)

    return d

In [6]:
def makefloat(d):
    for k in d.keys():
        for i in range(len(d[k]) - 1):
            d[k][i] = float(d[k][i])

    return d

In [7]:
def plot_delay_ov_distance(d, distance):
    x1 = []
    y1 = []

    for k in d.keys():
        x1.append(k)
        y1.append(d[k][0])
        # print(d[k][3])

    coef_1 = np.polyfit(x1,y1,1)
    print(coef_1)
    poly1d_fn_1 = np.poly1d(coef_1) 

    midval = poly1d_fn_1(1500)

    print(midval)
    
    for k in d.keys():
        # print(d[k])
        d[k][0] = float(d[k][0]) - midval
        if (d[k][0] != 0):
            d[k][3] = abs((((distance - 2*k) / d[k][0]) / 299792458)*10000000000)

    

    # d = offset(d)

    x2 = []
    y2 = []

    for k in d.keys():
        x2.append(k)
        y2.append(d[k][0])
        # print(d[k][3])

    coef_2 = np.polyfit(x2,y2,1)
    print(coef_2)
    poly1d_fn_2 = np.poly1d(coef_2) 
    # poly1d_fn is now a function which takes in x and returns an estimate for y

    plt.plot(x1, y1, 'bo', x2, y2, 'ro', x1, poly1d_fn_1(x1), '--y', x2, poly1d_fn_2(x2), '--k') #'--k'=black dashed line, 'yo' = yellow circle marker
    plt.xlabel("Distance from LED Pulse (mm)")
    plt.ylabel("Delay between SiPM activations, ns")
    plt.legend(["Original Data", "Adjusted Data"])
    plt.show()

In [8]:
def plot_sd_ov_distance(d):
    x1 = []
    y1 = []

    for k in d.keys():
        x1.append(k)
        y1.append(d[k][1])
        # print(d[k][3])

    coef_1 = np.polyfit(x1,y1,1)
    # print(coef_1)
    poly1d_fn_1 = np.poly1d(coef_1)
    # poly1d_fn is now a function which takes in x and returns an estimate for y


    plt.plot(x1, y1, 'bo')
    # plt.plot(x1, y1, 'bo', x1, poly1d_fn_1(x1), '--y') #'--k'=black dashed line, 'yo' = yellow circle marker
    plt.xlabel("Distance from LED Pulse (mm)")
    plt.ylabel("Standard Deviation of the Delay values, ns")
    plt.show()

In [9]:
def avg_SD(data):
    total = 0
    for key in data.keys():
        total += data[key][1]

    return total / len(data.keys())
    

In [11]:
d_old = histogram_justCalc('XaM_DEL_SGB-BCF-2-0_R', '.csv', [286, 838, 1127, 1422, 1715, 2000, 2286, 2581], 3000)

print(d_old)

d_old = makefloat(d_old)

plot_delay_ov_distance(d_old, 3000)

plot_sd_ov_distance(d_old)

avg_SD(d_old)

FileNotFoundError: [Errno 2] No such file or directory: 'XaM_DEL_SGB-BCF-1-2_R82.csv'